# import librareis

In [14]:
import pyspark as ps 
from pyspark.sql import functions as fun 

from pyspark.sql import SparkSession

from pyspark.sql.types import StructField, StringType, IntegerType, FloatType, StructType

In [2]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

c:\Users\INDHRNA\Desktop\ML\python\PySpark_Portilla\.venv\lib\site-packages\pyspark\context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


# paths

In [3]:
people = 'Spark_DataFrames\people.json'



# functions

In [7]:

df = spark.read.json(people)

df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [6]:
df.describe()

DataFrame[summary: string, age: string, name: string]

In [8]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [9]:
df.columns

['age', 'name']

In [10]:
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



## StructField

In [15]:
data_schema = [StructField('age',IntegerType(), nullable=True),
               StructField(name='name', dataType=StringType(), nullable=True)]
data_schema

[StructField('age', IntegerType(), True),
 StructField('name', StringType(), True)]

In [16]:
final_structure = StructType(fields=data_schema)
final_structure


StructType([StructField('age', IntegerType(), True), StructField('name', StringType(), True)])

In [18]:
new_df = spark.read.json(people, schema=final_structure)
new_df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



# Lecture 26 : Dataframe basics part TWO

In [19]:
type(new_df['age'])

pyspark.sql.column.Column

In [20]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [21]:
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [22]:
df.show(2)

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
+----+-------+
only showing top 2 rows



In [24]:
type(df.head(2)[0])

pyspark.sql.types.Row

In [25]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [27]:
df.withColumn('age_new',df['age']+10).show()

+----+-------+-------+
| age|   name|age_new|
+----+-------+-------+
|null|Michael|   null|
|  30|   Andy|     40|
|  19| Justin|     29|
+----+-------+-------+



In [28]:
df.withColumnRenamed('age','my_new_age').show()

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



In [32]:
df.createOrReplaceTempView('people')

In [33]:
results = spark.sql("select * from people")

In [34]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [35]:
new_result = spark.sql("select * from people where age=30")
new_result.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



# Lecture 3: Basic Operations

In [38]:
apple_stock = r'Spark_DataFrames\appl_stock.csv'
df = spark.read.csv(apple_stock,header=True,inferSchema=True)
df.show(3)

+----------+----------+----------+------------------+----------+---------+------------------+
|      Date|      Open|      High|               Low|     Close|   Volume|         Adj Close|
+----------+----------+----------+------------------+----------+---------+------------------+
|2010-01-04|213.429998|214.499996|212.38000099999996|214.009998|123432400|         27.727039|
|2010-01-05|214.599998|215.589994|        213.249994|214.379993|150476200|27.774976000000002|
|2010-01-06|214.379993|    215.23|        210.750004|210.969995|138040000|27.333178000000004|
+----------+----------+----------+------------------+----------+---------+------------------+
only showing top 3 rows



In [39]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [40]:
df.head(3)[0]

Row(Date=datetime.date(2010, 1, 4), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)

In [41]:
df.filter('close < 500').show(3)

+----------+----------+----------+------------------+----------+---------+------------------+
|      Date|      Open|      High|               Low|     Close|   Volume|         Adj Close|
+----------+----------+----------+------------------+----------+---------+------------------+
|2010-01-04|213.429998|214.499996|212.38000099999996|214.009998|123432400|         27.727039|
|2010-01-05|214.599998|215.589994|        213.249994|214.379993|150476200|27.774976000000002|
|2010-01-06|214.379993|    215.23|        210.750004|210.969995|138040000|27.333178000000004|
+----------+----------+----------+------------------+----------+---------+------------------+
only showing top 3 rows



In [43]:
df.filter('close < 300').select(['Open','Close']).show(4)


+----------+----------+
|      Open|     Close|
+----------+----------+
|213.429998|214.009998|
|214.599998|214.379993|
|214.379993|210.969995|
|    211.75|    210.58|
+----------+----------+
only showing top 4 rows



In [44]:
df.filter(df['Close']<500).select('Open').show(3)

+----------+
|      Open|
+----------+
|213.429998|
|214.599998|
|214.379993|
+----------+
only showing top 3 rows



In [47]:
df.filter((df['Close']<300) & (df['Open']>289)).show(3)

+----------+----------+------------------+----------+------------------+---------+---------+
|      Date|      Open|              High|       Low|             Close|   Volume|Adj Close|
+----------+----------+------------------+----------+------------------+---------+---------+
|2010-09-24| 292.10001|293.53000299999997|290.549988|292.31998799999997|162372000|37.872846|
|2010-09-27|    293.98|294.73000299999995|291.009998|        291.159988|120708700|37.722557|
|2010-09-28|291.769989|        291.769989|274.999992|        286.859997|258760600|37.165452|
+----------+----------+------------------+----------+------------------+---------+---------+
only showing top 3 rows



In [54]:
res1 = df.filter(df['Open']==293.98)
res1.show()

+----------+------+------------------+----------+----------+---------+---------+
|      Date|  Open|              High|       Low|     Close|   Volume|Adj Close|
+----------+------+------------------+----------+----------+---------+---------+
|2010-09-27|293.98|294.73000299999995|291.009998|291.159988|120708700|37.722557|
+----------+------+------------------+----------+----------+---------+---------+



In [51]:
result = df.filter((df['Open'] == 293.98)).collect()
result

[Row(Date=datetime.date(2010, 9, 27), Open=293.98, High=294.73000299999995, Low=291.009998, Close=291.159988, Volume=120708700, Adj Close=37.722557)]

In [52]:
row = result[0]
row

Row(Date=datetime.date(2010, 9, 27), Open=293.98, High=294.73000299999995, Low=291.009998, Close=291.159988, Volume=120708700, Adj Close=37.722557)

## asDict

In [53]:
row.asDict()['Volume']

120708700

# GroupBy and Aggregrate functions

In [55]:
sales_inf = r'Spark_DataFrames\sales_info.csv'

df = spark.read.csv(sales_inf, header=True, inferSchema=True)
df.show(9)

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
+-------+-------+-----+
only showing top 9 rows



In [56]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [57]:
df.groupBy('Company')

In [59]:
df.groupBy('Company').count().show()

+-------+-----+
|Company|count|
+-------+-----+
|   APPL|    4|
|   GOOG|    3|
|     FB|    2|
|   MSFT|    3|
+-------+-----+



In [60]:
df.groupBy('Person').max().show()

+-------+----------+
| Person|max(Sales)|
+-------+----------+
|  Linda|     130.0|
|    Amy|     124.0|
|  Sarah|     350.0|
|Charlie|     120.0|
|  Chris|     350.0|
|   John|     250.0|
|    Sam|     200.0|
|   Carl|     870.0|
|Vanessa|     243.0|
|   Mike|     750.0|
|   Tina|     600.0|
|  Frank|     340.0|
+-------+----------+



In [61]:
df.agg({'Sales':'sum'}).show()

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [65]:
df.agg({'Person':'count'}).show()

+-------------+
|count(Person)|
+-------------+
|           12|
+-------------+



In [66]:
from pyspark.sql import functions as fun 


In [68]:
df.select(fun.countDistinct('Person')).show()


+----------------------+
|count(DISTINCT Person)|
+----------------------+
|                    12|
+----------------------+



In [72]:
sales_std = df.select(fun.stddev('Sales').alias('std_dev'))
sales_std.show()

+------------------+
|           std_dev|
+------------------+
|250.08742410799007|
+------------------+



In [70]:
from pyspark.sql.functions import format_number

In [74]:
sales_std.select(format_number('std_dev',2)).show()

+-------------------------+
|format_number(std_dev, 2)|
+-------------------------+
|                   250.09|
+-------------------------+



In [75]:
df.orderBy('Sales').show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [76]:
df.orderBy(df['Sales'].desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+



# Missing Data

In [77]:
null_data = r'Spark_DataFrames\ContainsNull.csv'

df = spark.read.csv(null_data, header=True, inferSchema=True)
df.head(3)

[Row(Id='emp1', Name='John', Sales=None),
 Row(Id='emp2', Name=None, Sales=None),
 Row(Id='emp3', Name=None, Sales=345.0)]

In [78]:
df.show(3)

+----+----+-----+
|  Id|Name|Sales|
+----+----+-----+
|emp1|John| null|
|emp2|null| null|
|emp3|null|345.0|
+----+----+-----+
only showing top 3 rows



In [79]:
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [80]:
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [81]:
df.na.drop(subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [82]:
df.na.fill('').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2|     | null|
|emp3|     |345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [85]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [86]:
df.na.fill(0).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [87]:
df.na.fill(0,subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [88]:
from pyspark.sql.functions import mean

In [89]:
mean_val = df.select(mean(df['Sales'])).collect()
mean_val

[Row(avg(Sales)=400.5)]

In [97]:
mean_sales=mean_val[0][0]
mean_sales

400.5

In [96]:
df.select(mean(df['Sales'])).collect()

[Row(avg(Sales)=400.5)]

In [98]:
df.na.fill(mean_sales,subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



# Date time

In [100]:
apple_stock
df = spark.read.csv(apple_stock,header=True, inferSchema=True)
df.show(3)

+----------+----------+----------+------------------+----------+---------+------------------+
|      Date|      Open|      High|               Low|     Close|   Volume|         Adj Close|
+----------+----------+----------+------------------+----------+---------+------------------+
|2010-01-04|213.429998|214.499996|212.38000099999996|214.009998|123432400|         27.727039|
|2010-01-05|214.599998|215.589994|        213.249994|214.379993|150476200|27.774976000000002|
|2010-01-06|214.379993|    215.23|        210.750004|210.969995|138040000|27.333178000000004|
+----------+----------+----------+------------------+----------+---------+------------------+
only showing top 3 rows



In [101]:
df.select(['Date','Open']).show()

+----------+------------------+
|      Date|              Open|
+----------+------------------+
|2010-01-04|        213.429998|
|2010-01-05|        214.599998|
|2010-01-06|        214.379993|
|2010-01-07|            211.75|
|2010-01-08|        210.299994|
|2010-01-11|212.79999700000002|
|2010-01-12|209.18999499999998|
|2010-01-13|        207.870005|
|2010-01-14|210.11000299999998|
|2010-01-15|210.92999500000002|
|2010-01-19|        208.330002|
|2010-01-20|        214.910006|
|2010-01-21|        212.079994|
|2010-01-22|206.78000600000001|
|2010-01-25|202.51000200000001|
|2010-01-26|205.95000100000001|
|2010-01-27|        206.849995|
|2010-01-28|        204.930004|
|2010-01-29|        201.079996|
|2010-02-01|192.36999699999998|
+----------+------------------+
only showing top 20 rows



In [102]:
from pyspark.sql.functions import (dayofmonth,hour,dayofyear,year,
                                   month,weekofyear,format_number,date_format)

In [103]:
df.select(dayofmonth(df['Date'])).show()

+----------------+
|dayofmonth(Date)|
+----------------+
|               4|
|               5|
|               6|
|               7|
|               8|
|              11|
|              12|
|              13|
|              14|
|              15|
|              19|
|              20|
|              21|
|              22|
|              25|
|              26|
|              27|
|              28|
|              29|
|               1|
+----------------+
only showing top 20 rows



In [105]:
df.select(year(df['Date'])).show()

+----------+
|year(Date)|
+----------+
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
|      2010|
+----------+
only showing top 20 rows



In [107]:
newdf = df.withColumn('year',year(df['Date']))#.show()
newdf.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+----+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|year|
+----------+------------------+------------------+------------------+------------------+---------+------------------+----+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|2010|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|2010|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|2010|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|2010|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|2010|
|2010-01-11|212.

In [110]:
n1 = newdf.groupBy('Year').mean().select(['Year','avg(Close)'])
n1.show()

+----+------------------+
|Year|        avg(Close)|
+----+------------------+
|2015|120.03999980555547|
|2013| 472.6348802857143|
|2014| 295.4023416507935|
|2012| 576.0497195640002|
|2016|104.60400786904763|
|2010| 259.8424600000002|
|2011|364.00432532142867|
+----+------------------+



In [111]:
n1 = n1.withColumnRenamed('avg(Close)', 'avg_close')
n1.show()

+----+------------------+
|Year|         avg_close|
+----+------------------+
|2015|120.03999980555547|
|2013| 472.6348802857143|
|2014| 295.4023416507935|
|2012| 576.0497195640002|
|2016|104.60400786904763|
|2010| 259.8424600000002|
|2011|364.00432532142867|
+----+------------------+



In [112]:
n1.select(['Year',format_number(n1['avg_close'],2).alias('average close')]).show()

+----+-------------+
|Year|average close|
+----+-------------+
|2015|       120.04|
|2013|       472.63|
|2014|       295.40|
|2012|       576.05|
|2016|       104.60|
|2010|       259.84|
|2011|       364.00|
+----+-------------+

